In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FormatStrFormatter
import matplotlib.patches as mpatches

width = 505.89

inputnum_arr = np.array([1,2,3,4])
blknum_arr = np.array([10,20,50,100,200,500,1000])
datasz_arr = np.array([8192,65536,524288,4194304])
datasz_labels = np.array(["8KB", "64KB", "512KB", "4MB"])

numctxt_arr = np.array([10,20,50,100,200,500,1000])

In [ ]:
def set_size(width, fraction=1):
    """ Set aesthetic figure dimensions to avoid scaling in latex.
    Parameters
    ----------
    width: float
            Width in pts
    fraction: float
            Fraction of the width which you wish the figure to occupy
    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    # Width of figure
    fig_width_pt = width * fraction

    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio

    fig_dim = (fig_width_in, fig_height_in)

    return fig_dim

In [ ]:
def plot_kv(input_path, output_path):
    # VERIFY_KV (1 plot, bar graph, # of inputs on x-axis)
    w = 0.1
    maxw = w*3
    colors = ['darksalmon','burlywood','darkseagreen','cornflowerblue','thistle','rosybrown','silver']

    fig, ax = plt.subplots(1,1,figsize=set_size(width, fraction=1))

    input_data = pd.read_csv(input_path)
    x = np.arange(len(inputnum_arr))
    y_values = list()

    # Get latency values by block_num. lists in "values" is what we want to plot
    for blk_num in blknum_arr:
        y_values.append(input_data.loc[input_data['block_num'] == blk_num]['total'].tolist())
       
    for i in range(len(y_values)):
        ax.bar(x+(-maxw+i*w), y_values[i], w, color=colors[i])

    plt.xticks(x,inputnum_arr, fontsize=12)
    plt.yticks(fontsize=12)

    plt.xlabel("Number of dependencies", fontsize=12)
    plt.ylabel("Latency (s)", fontsize=12)
    
    plt.legend(blknum_arr.astype(str), title_fontsize=12, fontsize=12, title="Number of blocks", ncol=1, bbox_to_anchor=(1,0.8))
    plt.grid(linestyle='--', color='silver', axis='y', linewidth=0.5)
    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
v_kv_path = "./data/kv.csv"
plot_kv(v_kv_path, "./kv.pdf")

In [ ]:
def plot_kv_subplot(input_path, output_path, isLocal, isLog):
    fig, axs = plt.subplots(2,2,figsize=set_size(width, fraction=1))
    colors = ['coral','mediumseagreen', 'dodgerblue', 'darkorchid']

    input_data = pd.read_csv(input_path)
    y_values = list()

    for input_num in inputnum_arr:
        y_values.append(input_data.loc[input_data['input_num'] == input_num]['total'].tolist())

    print(y_values)
        
    if isLog:
        x = blknum_arr
        bar_w = 0.2 * np.array(x)
    else:
        x = np.arange(len(blknum_arr))
        bar_w = 0.3
        
    axs[0,0].bar(x,y_values[0],bar_w,color=colors[0])
    axs[0,0].set_title("Number of dependencies = 1", fontsize=10)
    axs[0,1].bar(x,y_values[1],bar_w,color=colors[1])
    axs[0,1].set_title("Number of dependencies = 2", fontsize=10)
    axs[1,0].bar(x,y_values[2],bar_w,color=colors[2])
    axs[1,0].set_title("Number of dependencies = 3", fontsize=10)
    axs[1,1].bar(x,y_values[3],bar_w,color=colors[3])
    axs[1,1].set_title("Number of dependencies = 4", fontsize=10)

    for ax in axs.flat:
        if isLog:
            ax.set_xscale('log')
        ax.set_xticks(x, blknum_arr, fontsize=10)
        ax.set_xlabel("Number of blocks", fontsize=10)
        ax.set_ylabel('Latency (s)', fontsize=10)
        ax.label_outer()
    
    if isLocal:
        axs[0,0].sharey(axs[0,1])
        yts = axs[1,1].get_yticks()
        yts = yts.astype(np.double)
        new_yts = np.arange(min(yts), max(yts), 0.1)
        axs[1,1].set_yticks(new_yts, new_yts, fontsize=10)
        axs[1,0].set_yticks(new_yts, new_yts, fontsize=10)
        axs[1,0].sharey(axs[1,1])
        axs[1,0].set_ylim(0,0.61)
        plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%g'))
    else:
        yts = axs[0,1].get_yticks()
        yts = yts.astype(np.double)
        new_yts = np.arange(min(yts), max(yts)+0.5, 0.5)
        axs[0,0].set_yticks(new_yts, new_yts, fontsize=10)
        axs[1,0].set_yticks(new_yts, new_yts, fontsize=10)
        axs[0,0].sharey(axs[0,1])

        yts = axs[1,1].get_yticks()
        yts = yts.astype(np.double)
        new_yts = np.arange(min(yts), max(yts), 0.5)
        axs[1,1].set_yticks(new_yts, new_yts, fontsize=10)
        axs[1,0].set_yticks(new_yts, new_yts, fontsize=10)
        axs[1,0].sharey(axs[1,1])

    for ax in axs.flat:
        ax.grid(linestyle='--', color='darkgrey', axis='y', linewidth=0.4)

    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
v_kv_path = "./data/kv.csv"

plot_kv_subplot(v_kv_path, "./kv_sub.pdf", False, False)
plot_kv_subplot(v_kv_path, "./kv_sub_log.pdf", False, True)

In [ ]:
def plot_opc(input_path, output_path):
    # VERIFY_OPC (1 plot, bar graph, # of inputs on x-axis)
    w = 0.1
    colors = ['darksalmon','burlywood','darkseagreen','cornflowerblue']

    fig, ax = plt.subplots(1,1,figsize=set_size(width, fraction=1))

    input_data = pd.read_csv(input_path)
    x = np.arange(len(datasz_arr))
    y_values = list()

    # Get latency values by block_num. lists in "values" is what we want to plot
    for ds in datasz_arr:
        y_values.append(input_data.loc[input_data['data_size'] == ds]['total'].tolist())

    print(y_values)

    for i in range(len(y_values)):
    #     ax.bar(x+(-0.2+i*w+0.05), y_values[i], w, color=colors[i])
        ax.plot(x, y_values[i], marker="o", color=colors[i])

    plt.xticks(x,inputnum_arr, fontsize=12)
    yts = ax.get_yticks().astype(np.int64)
    plt.yticks(yts, yts, fontsize=12)
    plt.ylim([0, yts[-1]])
    
    plt.xlabel("Number of dependencies", fontsize=12)
    plt.ylabel("Latency (s)", fontsize=12)
    plt.legend(datasz_labels.astype(str), fontsize=12, title="Data size", title_fontsize=12, ncol=1, bbox_to_anchor=(1.25,0.75))
    plt.grid(linestyle='--', color='silver', axis='y', linewidth=0.4)
    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
v_opc_path = "./data/opc.csv"
plot_opc(v_opc_path, "./opc.pdf")

In [ ]:
def plot_sign(input_path, output_path):
    # SIGN (1 plot, bar graph, # of inputs on x-axis)
    w = 0.1
    colors = ['darksalmon','burlywood','darkseagreen','cornflowerblue']

    fig, ax = plt.subplots(1,1,figsize=set_size(width, fraction=1))

    input_data = pd.read_csv(input_path)
    x = np.arange(len(datasz_arr))
    y_values = list()

    # Get latency values by block_num. lists in "values" is what we want to plot
    for ds in datasz_arr:
        y_values.append(input_data.loc[input_data['data_size'] == ds]['total'].tolist())

    print(y_values)

    for i in range(len(y_values)):
#         ax.bar(x+(-0.2+i*w+0.05), y_values[i], w, color=colors[i])
        ax.plot(x, y_values[i], marker="o", color=colors[i])


    ax.legend(datasz_labels.astype(str), fontsize=12, title="Data size", title_fontsize=12, ncol=1, bbox_to_anchor=(1.25,0.75))

    plt.xticks(x, inputnum_arr, fontsize=12)
    yts = ax.get_yticks().astype(np.int64)
    plt.yticks(yts, yts, fontsize=12)
    plt.ylim([0, yts[-1]])
    
    plt.xlabel("Number of outputs", fontsize=12)
    plt.ylabel("Latency (s)", fontsize=12)
    plt.grid(linestyle='--', color='silver', axis='y', linewidth=0.4)
    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
sign_path = "./data/sign.csv"
plot_sign(sign_path, "./sign.pdf")

In [ ]:
# evote_path = "./data/evote.csv"
# plot_evote(evote_path, "./evote.pdf")

In [ ]:
def plot_dfu_mb(reg_path, dfu_path, output_path):
    fig, ax = plt.subplots()
    
    reg_data = pd.read_csv(reg_path)
    dfu_data = pd.read_csv(dfu_path)

    x = numctxt_arr
    reg_values = reg_data['total'].tolist()
    dfu_values = dfu_data['total'].tolist()
    
    print(reg_values)
    print(dfu_values)
    
    ax.plot(x, reg_values, marker="o", color='royalblue', label='Regular')
    ax.plot(x, dfu_values, marker="o", color='palevioletred', label='Protean')

    plt.xscale('log')
    plt.xticks(x, numctxt_arr, fontsize=12)

    yts = ax.get_yticks().astype(np.int64)
    if "shuf" in reg_path:
        new_yts = np.arange(0, max(yts)+1, 10)
    else:
        new_yts = np.arange(0, max(yts)+1, 5)
    plt.yticks(new_yts, new_yts, fontsize=12)
    
    plt.xlabel("Number of ciphertexts", fontsize=12)
    plt.ylabel("Latency (s)", fontsize=12)
    plt.legend(fontsize=12,ncol=1,bbox_to_anchor=(1.3,0.6))
    plt.grid(linestyle='--', color='silver', axis='y', linewidth=0.4)
    
    if "shuf" in reg_path:
        plt.title("Shuffle protocol")
    else:
        plt.title("Threshold decryption protocol", fontsize=12)
    
    plt.savefig(output_path, bbox_inches='tight', format='pdf')

In [ ]:
plot_dfu_mb('./data/shuf_reg.csv', './data/shuf_dfu.csv', './shuf.pdf')
plot_dfu_mb('./data/dec_reg.csv', './data/dec_dfu.csv', './dec.pdf')

In [ ]:
# JOIN/VOTE

import csv

def process_participate(path, txn_type, app_type):
    data_read = list()
    with open(path, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        labels = next(reader)
        for row in reader:
            data_read.append(dict(zip(labels, row)))

    latency_vals = dict()
    for dr in data_read:
        num_participants = int(dr['numparticipants'])
        if num_participants > 5:
            vals = list()
            for i in range(num_participants):
                vals.append(float(dr[f'p{i}_{txn_type}_wall_avg']))
            latency_vals[num_participants] = vals

#     print("num_participants,avg,min,max")
    header = ['num_participants','avg','min','max']
    with open(f'./data/{app_type}/{app_type}_{txn_type}.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        for k in latency_vals:
            vals = np.array(latency_vals[k])
            min_val = vals.min()
            max_val = vals.max()
            mean_val = vals.mean()
            data = [num_participants, mean_val, min_val, max_val]
            writer.writerow(data)
            print("%d,%.6f,%.6f,%.6f" % (k, mean_val, min_val, max_val))
    

In [ ]:
# [RAND|DKG]LOTTERY_CLOSE
import numpy as np
import csv
import re

spattern = "setup_(\w+)_wall_avg"
lpattern = "lock_(\w+)_wall_avg"
shpattern = "shuffle_(\w+)_wall_avg"
tpattern = "tally_(\w+)_wall_avg"

cpattern = "close_(\w+)_wall_avg"
fpattern = "finalize_(\w+)_wall_avg"

header = ['num_participants', 'total']

def process_lottery(path, pattern, app_type):
    data_read = list()
    with open(path, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        labels = next(reader)
        for row in reader:
            data_read.append(dict(zip(labels, row)))

#     print("num_participants,total")
    tokens = pattern.split("_")
    txn_type = tokens[0]
    with open(f'./data/{app_type}/{app_type}_{txn_type}.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        data = dict()
        for dr in data_read:
            num_participants = int(dr['numparticipants'])
            if num_participants > 5:
                total = 0.0
                for label, value in dr.items():
                    match = re.search(pattern, label)
                    if match is not None and match.group(1) != "getstate":
                        total += float(value)
                data = [num_participants, total]
                writer.writerow(data)
                print("%d,%.6f" % (num_participants, total))

In [ ]:
app_type = 'randlottery'

print(">>> CLOSE")
process_lottery("../randlottery/test_data/randlottery_v0.csv", cpattern, app_type)

print(">>> FINALIZE")
process_lottery("../randlottery/test_data/randlottery_v0.csv", fpattern, app_type)

In [ ]:
app_type = 'randlottery'

print(">>> CLOSE")
process_lottery("../randlottery/test_data/randlottery_v1.csv", cpattern, app_type)

print(">>> FINALIZE")
process_lottery("../randlottery/test_data/randlottery_v1.csv", fpattern, app_type)

In [ ]:
app_type = 'dkglottery'

print(">>> CLOSE")
process_lottery("../dkglottery/test_data/dkglottery_v0.csv", cpattern, app_type)

print(">>> FINALIZE")
process_lottery("../dkglottery/test_data/dkglottery_v0.csv", fpattern, app_type)

In [ ]:
app_type = 'randlottery'
process_participate("../randlottery/test_data/randlottery_v0.csv", "join", app_type)

In [ ]:
app_type = 'randlottery'
process_participate("../randlottery/test_data/randlottery_v1.csv", "join", app_type)

In [ ]:
app_type = 'dkglottery'
process_participate("../dkglottery/test_data/dkglottery_v0.csv", "join", app_type)